This notebook covers testing of the Unstructured library for parsing documents as part of the MicroRAG experiment. The goal of Microrag is to test the ability of retrieval augmented generation (RAG) to produce specialized engineering reports. Since I have a couple degrees in geotechnical engineering, I thought it would be nice to try and produce something useful for my faculty, so let's start there.

We are going to use a Raw - Refined - Produced data extraction pattern

In [ ]:
from pathlib import Path
from unstructured.partition.auto import partition

raw_geotech_reports_path = Path('/home/smckean/Raw/geotechical_reports')
report = next(raw_geotech_reports_path.glob('*.pdf'))

print(report.name)
elements = partition(filename=report)
print("\n\n".join([str(el) for el in elements]))

In [18]:
elements[518].text

'PROJECT NO.: RD2894 BH LOCATION:'

In [21]:
from unstructured.partition.pdf import partition_pdf_or_image

scanned_doc = raw_geotech_reports_path / '061105_ISSC002GeotechnicalInvestigationSpec_PetroCanada.pdf'
elements = partition_pdf_or_image(scanned_doc, languages=['eng'])

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

from unstructured.documents.elements import NarrativeText
from unstructured.staging.huggingface import stage_for_transformers

model_name = "hf-internal-testing/tiny-bert-for-token-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
tf_staged = stage_for_transformers(elements, tokenizer)

/home/smckean/Repos/microrag/microrag_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
from unstructured.chunking.basic import chunk_elements
chunk_elements(elements)[8].text

'The report shall include all field data, laboratory test results, and the resulting interpretations and recommendations. All testing, interpretation of results and recommendations shall be completed under the direct supervision of a practicing Professional Engineer registered in the Province of Alberta. 3.0 LOCATION (GENERAL) See SPS-C-002 for specific project details. 4.0 DRILLING AND SAMPLING'

In [ ]:
# Use this to store the document as a json file in refined (to reproduce the process)
from unstructured.staging.base import convert_to_dict
convert_to_dict(elements)

In [53]:
from unstructured.embed.huggingface import HuggingFaceEmbeddingEncoder, HuggingFaceEmbeddingConfig
config = HuggingFaceEmbeddingConfig(model_name="bert-base-uncased")
embedder = HuggingFaceEmbeddingEncoder(config)
embedded = embedder.embed_documents(elements)

No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.
/home/smckean/Repos/microrag/microrag_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [54]:
len(embedded[0].embeddings)

768